In [5]:
import time

import vk
from pyvis.network import Network

In [6]:
token = 'vk1.a.yDBY6lSHQbHsWUTSvPuZx9oL_huBPwJBiwQMoJ0mQ1z-Zh_PNf3rGvkXDS1nD2W8T00YKtnne5uafFkl4UPofb8tjbZOBWrVeZPsj8h4WUP57OeUWe2Q_EXkdJtsDEZPPvL7kltAy_xjJTxezPkbyajx6dCbpdq1NMmDzBk_vogTdAOZ23KYgdRUSv03_tzBT0d46syrGdjk9Z3HXdccow'
vk_api = vk.API(token, v='5.131')

In [7]:
max_friends_per_friend = 100
def get_user_graph(user_id):
    friends = vk_api.friends.get(user_id=user_id)['items']
    friends_of_friends_dict = {}
    friends_set = set()
    friends_edges_set = set()

    for friend in friends:
        try:
            friends_of_friends = vk_api.friends.get(user_id=friend)['items']
        except Exception:
            continue # если пользователь удален
        if len(friends_of_friends) > max_friends_per_friend:
            continue
        friends_of_friends_dict[friend] = friends_of_friends
        
        friends_set.add(friend)
        for friend_of_friend in friends_of_friends_dict[friend]:
            friends_set.add(friend_of_friend)
            friend_pair = [friend, friend_of_friend]
            friend_pair.sort()
            friends_edges_set.add(tuple(friend_pair))
        time.sleep(0.1) # потому что ограничение на количество запросов в секунду

    return friends_set, friends_edges_set

In [8]:
id_anton = 243317609
id_matvey = 207227130

anton_friend_nodes, anton_friend_edges = get_user_graph(id_anton)
matvey_friend_nodes, matvey_friend_edges = get_user_graph(id_matvey)

In [9]:
friend_set = anton_friend_nodes.union(matvey_friend_nodes)
friend_edge_set = anton_friend_edges.union(matvey_friend_edges)

In [10]:
graph = Network(notebook=True)

for friend in friend_set:
    graph.add_node(friend, label=str(friend))

for friend_pair in friend_edge_set:
    graph.add_edge(friend_pair[0], friend_pair[1])

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [11]:
graph.show("graph.html")